Install Requirements ()

In [1]:
!pip install autogen-agentchat autogen-ext[openai] -q
!pip install groq -q
!pip install arxiv -q
!pip install PyPDF2 -q
!pip install pymupdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.0/234.0 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 66.9 MB/s eta 0:00:00


In [2]:
!pip install transformers torch -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.2 MB/s eta 0:00:00


In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.tools import FunctionTool
from autogen_agentchat.conditions import MaxMessageTermination
import arxiv

In [4]:
import os
from getpass import getpass

# tokenGROQ = getpass('Enter GROQ_API_KEY here: ')
os.environ["GROQ_API_KEY"] = "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"

In [26]:
import arxiv
import fitz  # PyMuPDF
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline
import re

# Ensure necessary nltk data is downloaded
nltk.download("punkt")

# Load an improved summarization model (BART for better performance)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def extract_text_from_pdf(pdf_path):
    """Extracts text from a given PDF file and removes everything after 'References' in any form."""
    doc = fitz.open(pdf_path)
    text = "".join(page.get_text() + "\n" for page in doc)

    # Remove everything after the word "References" in any case variation
    text = re.split(r'(?i)\breferences\b', text, maxsplit=1)[0]

    return text

def chunk_text(text, chunk_size=1024):
    """Splits text into larger chunks for better summarization."""
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) < chunk_size:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def summarize_chunks(chunks):
    """Summarizes each chunk using BART for better quality summaries."""
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=150, min_length=50, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    return " ".join(summaries)  # Combine all summarized parts

def download_and_summarize_papers(query, num_papers=1):
    """Downloads papers, extracts text, chunks them, and summarizes the content."""
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=num_papers,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )

    paper_summaries = []

    for result in client.results(search):
        filename = f"{result.get_short_id()}.pdf"
        result.download_pdf(filename=filename)
        print(f"Downloaded: {filename}")

        text = extract_text_from_pdf(filename)
        chunks = chunk_text(text)
        summary = summarize_chunks(chunks)

        paper_summaries.append({"id": result.get_short_id(), "summary": summary})

    return paper_summaries

# Example usage:
papers = download_and_summarize_papers("Multi-agent LLM systems", num_papers=1)
for paper in papers:
    print(f"Paper ID: {paper['id']}, Summary: {paper['summary']}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cuda:0


Downloaded: 2503.10630v1.pdf
Paper ID: 2503.10630v1, Summary: In this paper, we propose a general framework for univer-sal zero-shot goal-oriented navigation. We propose a uniform graph representation to unify different goals. We also convert the observation of agent into an online maintained scene graph. We preserve most structural infor-                mation compared with pure text. We are able to leverage GLM for explicit graph-based reasoning. Extensive experiments on several benchmarks show that our UniGoal achieves state-of-the-art zero-shot performance on three studied navigation tasks. Goal-oriented navigation is a fundamental problem in robotic tasks. It requires the agent to navigate to a specified goal in an unknown environment. State-of-the-art zero-shot goal-oriented Navigation meth-phthalods are typically specialized for each goal type. UniGoal enables zero-shotinference on three studied navigation tasks. It achieves leading performance on multiple benchmarks. These sub-

In [28]:
print(papers[0].get("summary"))

In this paper, we propose a general framework for univer-sal zero-shot goal-oriented navigation. We propose a uniform graph representation to unify different goals. We also convert the observation of agent into an online maintained scene graph. We preserve most structural infor-                mation compared with pure text. We are able to leverage GLM for explicit graph-based reasoning. Extensive experiments on several benchmarks show that our UniGoal achieves state-of-the-art zero-shot performance on three studied navigation tasks. Goal-oriented navigation is a fundamental problem in robotic tasks. It requires the agent to navigate to a specified goal in an unknown environment. State-of-the-art zero-shot goal-oriented Navigation meth-phthalods are typically specialized for each goal type. UniGoal enables zero-shotinference on three studied navigation tasks. It achieves leading performance on multiple benchmarks. These sub-tasks are also known as Object-goalNavigation (ON) and Instanc

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_core.tools import FunctionTool
from autogen_agentchat.teams import SelectorGroupChat
from typing import List

text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=7)
combined_termination = max_messages_termination | text_termination

custom_model_client = OpenAIChatCompletionClient(
    model="llama-3.1-8b-instant",
    # model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

def get_summary():
    global summary
    return summary

get_summary_tool = FunctionTool(
    get_summary,
    description="Retrieves the summary of the research."
)

report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=custom_model_client,
    tools=[get_summary_tool],
    description="Generates a high-quality literature review based on the provided summary.",
    system_message="""
    You are the Report Agent. Your task is to synthesize the provided summary into a high-quality literature review.
    Use the `get_summary_tool` to retrieve the summary and write a well-structured review.
    """
)

planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent responsible for planning and coordinating the workflow of the research team.",
    model_client=custom_model_client,
    system_message="""
    You are the Planning Agent. Your responsibilities are:
    1. Initiate the review process by asking the `Report_Agent` to generate a literature review.
    2. Present the review to the user and collect their feedback.
    3. If the user provides feedback, ask the `Report_Agent` to revise the review.
    4. Repeat the process until the user is satisfied or terminates the workflow.

    Follow these steps:
    1. Start by asking the `Report_Agent` to generate a literature review using its tool.
    2. Once the review is generated, present it to the user and ask for feedback.
    3. If the user provides feedback, pass it to the `Report_Agent` to revise the review.
    4. Repeat steps 2-3.

    Your role is to ensure the workflow runs smoothly and that the user's feedback is incorporated into the review.
    """
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    input_func=input
)


team = SelectorGroupChat(
    [planning_agent, report_agent, user_proxy],
    termination_condition=combined_termination,
    model_client=custom_model_client,
    allow_repeated_speaker=True,
    selector_prompt=(
        "Available roles:\n{roles}\nTheir job descriptions:\n{participants}\n"
        "Current conversation history:\n{history}\n"
        "Please select the most appropriate role for the next message, and only return the role name."
    ),
)

await Console(
    team.run_stream(
        task="Write a literature review",
    )
)

In [27]:
!pip install numpy -q
!pip install nltk -q
!pip install rouge -q
!pip install bert_score -q
!pip install textstat -q
!pip install spacy -q
!pip install torch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.2 MB/s eta 0:00:00


In [29]:
import numpy as np
import nltk
import rouge
import bert_score
import textstat
import spacy
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu

# Download required resources
nltk.download('punkt')
nlp = spacy.load("en_core_web_sm")

# Load GPT-2 for perplexity calculation
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

def compute_rouge_scores(reference, generated):
    evaluator = rouge.Rouge()
    scores = evaluator.get_scores(generated, reference)
    return scores

def compute_bleu(reference, generated):
    reference_tokens = [nltk.word_tokenize(reference)]
    generated_tokens = nltk.word_tokenize(generated)
    return sentence_bleu(reference_tokens, generated_tokens)

def compute_bert_score(reference, generated):
    P, R, F1 = bert_score.score([generated], [reference], lang="en")
    return {'precision': P.mean().item(), 'recall': R.mean().item(), 'f1': F1.mean().item()}

def compute_readability(text):
    return textstat.flesch_reading_ease(text)

def compute_perplexity(text):
    tokens = gpt2_tokenizer.encode(text, return_tensors="pt")
    with torch.no_grad():
        loss = gpt2_model(tokens, labels=tokens).loss
    return torch.exp(loss).item()

def compute_cosine_similarity(reference, generated):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([reference, generated])
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    return similarity

def extract_cited_entities(text):
    doc = nlp(text)
    citations = [ent.text for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'WORK_OF_ART']]
    return set(citations)

def evaluate_reviews(reference_text, generated_text):
    scores = {}

    scores['rouge'] = compute_rouge_scores(reference_text, generated_text)
    scores['bleu'] = compute_bleu(reference_text, generated_text)
    scores['bert_score'] = compute_bert_score(reference_text, generated_text)
    scores['cosine_similarity'] = compute_cosine_similarity(reference_text, generated_text)
    scores['readability'] = compute_readability(generated_text)
    scores['perplexity'] = compute_perplexity(generated_text)

    reference_citations = extract_cited_entities(reference_text)
    generated_citations = extract_cited_entities(generated_text)
    scores['citation_overlap'] = len(reference_citations & generated_citations) / max(1, len(reference_citations))

    return scores

reference_text = """
ordinate Projection: Preserves spatial relationships for improved object localization.

4. Experimental Results
UniGoal outperforms state-of-the-art methods on MP3D, HM3D, and RoboTHOR, surpassing SG-Nav and Mod-IIN. Ablation studies confirm the effectiveness of its exploration strategy.

5. Conclusion
UniGoal advances zero-shot navigation by unifying graph-based goal representation and multi-stage exploration, outperforming existing methods and paving the way for further improvements in robotic navigation.

"""
generated_text = """
1. **Summary accuracy:** The summary is accurate and well-written. It effectively captures the main points of the paper, including the UniGoal framework, its graph-based representation, graph-based reasoning system, and impressive performance on various benchmarks.

2. **Key points:** The five key points listed are relevant and useful. However, I would suggest rephrasing or expanding on points 2 and 4 to better explain the graph-based reasoning system and generalization ability.

Consider rewriting point 2 to something like: "Graph-based reasoning: UniGoal's graph-based reasoning system enables it to identify similarities and relationships between the scene graph and goal graph, allowing it to reason about the scene and goal in a flexible and efficient manner."

For point 4, consider rephrasing it to: "Generalization ability: UniGoal's ability to represent the scene and goal uniformly enables it to adapt to a wide range of environments and goal types, making it effective in handling complex navigation tasks with multiple objects and relationships."

3. **Expansion on specific sections:** To further enhance the literature review, I would suggest expanding on the section discussing the graph-based representation and graph-based reasoning system. Providing more detail on how these components work together to achieve excellent performance on various navigation tasks would help readers better understand the paper's contributions.

Additionally, consider adding more context or discussion on the significance of the paper's findings, such as the implications for robotics, AI, or other relevant fields.

Overall, the literature review provides a clear and concise summary of the paper's main contributions. With a few minor adjustments, it can be even more effective in conveying the paper's importance and breakthroughs to readers.

"""
results = evaluate_reviews(reference_text, generated_text)
print(results)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


{'rouge': [{'rouge-1': {'r': 0.2962962962962963, 'p': 0.10062893081761007, 'f': 0.15023473799907439}, 'rouge-2': {'r': 0.01639344262295082, 'p': 0.004219409282700422, 'f': 0.006711406140040316}, 'rouge-l': {'r': 0.25925925925925924, 'p': 0.0880503144654088, 'f': 0.13145539527606967}}], 'bleu': 2.3493764478189283e-155, 'bert_score': {'precision': 0.8187416791915894, 'recall': 0.8446012735366821, 'f1': 0.8314704298973083}, 'cosine_similarity': 0.3479110697118061, 'readability': 21.63, 'perplexity': 19.01231575012207, 'citation_overlap': 0.0}
